In [2]:
import pandas as pd
import glob  
import sys
import os
import time
import re
import numpy as np
from tqdm import tqdm

# Get Data with filtering

Get keyword list

In [3]:
keyword_list = pd.read_excel("../data/Text_keyword_0620.xlsx")

In [4]:
print(len(keyword_list))
keyword_list.head()

65


,Unnamed: 0,Target 키워드
0,1,인구
1,2,도시
2,3,도심
3,4,지역
4,5,지자체


In [5]:
DATA_PATH = '../data/pkl/'


df_filter = pd.DataFrame()

df_len = []
city_list = []
total_before_len = 0

for f in tqdm(glob.glob(f'{DATA_PATH}*.pkl')):
    if '시' in f:
        city = re.search(f'{DATA_PATH}(.+?)_merged.pkl', f).group(1)
        df = pd.read_pickle(f)
        df['city'] = city
        city_list.append(city)
        city_before_len = len(df)
        total_before_len += city_before_len
        filter_keyword = []
        for idx, keyword in enumerate(df['키워드']):
            keyword =  str(keyword).split(',')
            filter_keyword.append(set(keyword).intersection(set(keyword_list['Target 키워드'])))
        df['filter_keyword'] = filter_keyword
        filtered_df = df.loc[df['filter_keyword'].map(len) != 0]
        #print(f"City = {city}, before len = {city_before_len}, after len = {len(filtered_df)}")
        df_filter = pd.concat([df_filter, filtered_df])        
        temp = [city,  city_before_len]
        df_len.append(temp)
pd.DataFrame(df_len, columns=['city', 'count']).to_csv("../result/before_filter_city_count.csv")

#print(f"before : {total_before_len}, after : {len(df_filter)}")

 44%|████▍     | 36/81 [02:27<04:06,  5.48s/it]

In [ ]:
df_filter.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', 'city', 'filter_keyword'],
      dtype='object')

### 분석제외 여부에 따른 filtering

In [ ]:
df_filter['분석제외 여부'].value_counts()

중복        178497
예외        134544
중복, 예외      4648
Name: 분석제외 여부, dtype: int64

In [ ]:
df_filter.shape

(6305834, 21)

In [ ]:
df_filter2 = df_filter[df_filter['분석제외 여부'].isnull()]

In [ ]:
print(df_filter2['분석제외 여부'].value_counts())
df_filter2.shape

Series([], Name: 분석제외 여부, dtype: int64)


(5988145, 21)

### 위치 정보에 너무 많은 도시가 등장하면 제거하기

In [ ]:
df_filter2.reset_index(inplace=True, drop = True)

In [ ]:
def find_general_news(article):
    related_city = []
    article = str(article).split(',')
    city = [x for i in article for x in city_list if i in x]
    related_city.append(city)
    return related_city


In [ ]:
related_city = df_filter2['위치'].apply(lambda x : find_general_news(x))

In [ ]:
df_filter2['related_city'] = related_city

/tmp/ipykernel_23604/3785826648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter2['related_city'] = related_city


In [ ]:
idx_list = []
for idx, city in enumerate(related_city):
    if len(city[0]) <5:
        idx_list.append(idx)

In [ ]:
df_filter3 = df_filter2.iloc[idx_list]

In [ ]:
len(df_filter3)

5246886

### 통합 분류1을 기준으로 필터링

In [ ]:
df_filter3.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부', 'city', 'filter_keyword',
       'related_city'],
      dtype='object')

In [ ]:
df_filter3['통합 분류1'].value_counts()

지역>강원            419483
지역>충남            418902
지역>경기            410381
지역>경남            405968
지역>충북            282363
                  ...  
스포츠>축구>국가대표팀        385
경제>외환               210
스포츠>야구>메이저리그        106
스포츠>야구>일본프로야구        20
006000000             1
Name: 통합 분류1, Length: 87, dtype: int64

In [ ]:
sum(df_filter3['통합 분류1'] == '정치>북한')
sum(df_filter3['통합 분류1'] == '정치>국회_정당')
sum(df_filter3['통합 분류1'] == '정치>선거')
sum(df_filter3['통합 분류1'] == '정치>외교')

7383

In [ ]:
'국제>예시' == re.compile("^국제")

False

In [ ]:
import re

idx_lst = []
for idx, cat in enumerate(df_filter3['통합 분류1']):
    if re.compile('^국제').findall(cat):
        continue
    elif re.compile('^스포츠').findall(cat):
        continue
    elif re.compile('^정치>북한').findall(cat):
        continue
    elif re.compile('^정치>국회_정당').findall(cat):
        continue
    elif re.compile('^정치>선거').findall(cat):
        continue
    elif re.compile('^정치>외교').findall(cat):
        continue
    else:
        idx_lst.append(idx)

In [ ]:
df_filter4 = df_filter3.iloc[idx_lst]

In [ ]:
len(df_filter4)

4964640

### Count # by city

In [ ]:
df_after_filter_len = pd.DataFrame(df_filter4['city'].value_counts())

In [ ]:
df_after_filter_len.reset_index(inplace = True)

In [ ]:
df_len = pd.DataFrame(df_len)

In [ ]:
df_len.columns = ['city', 'before']
df_after_filter_len.columns = ['city', 'after']

In [ ]:
city_count = pd.merge(df_len, df_after_filter_len)

In [ ]:
city_count['reduced ratio'] = city_count['after'] / city_count['before']

In [ ]:
city_count.to_csv("../result/city_count_before_and_after.csv")

: 

save

In [ ]:
print(df_filter.shape)
df_filter.to_pickle(f"{DATA_PATH}merged_filtered_0630.pkl")
print('done well')

(6305834, 21)
